In [3]:
#des function

class mangler_func():
    
    #initialisation    
    def __init__(self,plain_32bit,key_48bit):
        self.plain_32bit = plain_32bit
        self.key_48bit = key_48bit
        self.sbox_list = list()
        self.final32bit = str()
        
        self.expansion_box = [32,1,2,3,4,5,4,5,6,7,8,9,8,9,10,11,
                              12,13,12,13,14,15,16,17,16,17,18,19,20,21,20,21,
                              22,23,24,25,24,25,26,27,28,29,28,29,30,31,32,1]
    
        self.straight_box = [16,7,20,21,29,12,28,17,1,15,23,26,5,18,31,10,
                               2,8,24,14,32,27,3,9,19,13,30,6,22,11,4,25]

        self.s_box = [
            # Box-1
            [
                [14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
                [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
                [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
                [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]
            ],
            # Box-2
            [
                [15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
                [3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
                [0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
                [13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]
            ],
            # Box-3
            [
                [10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
                [13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
                [13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
                [1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]
            ],
            # Box-4
            [
                [7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
                [13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
                [10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
                [3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]
            ],
            # Box-5
            [
                [2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
                [14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
                [4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
                [11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]
            ],
            # Box-6
            [
                [12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
                [10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
                [9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
                [4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]
            ],
            # Box-7
            [
                [4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
                [13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
                [1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
                [6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]
            ],
            # Box-8
            [
                [13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
                [1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
                [7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
                [2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]
            ]
        ]
        
    #---------------------------
    def expansion48(self):
        expan_48bit = ''
        for index in self.expansion_box:
            expan_48bit = expan_48bit + self.plain_32bit[index-1]     
        return expan_48bit    
        
    def XOR(self,bit1,bit2):
        xor_result = ''
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r)
        return xor_result    
    
    def split_in_6bit(self,bit48):
        list_6bits = [(bit48[i:i+6]) for i in range(0,len(bit48),6)]
        return list_6bits
    
    def get_1and6bit(self,bit6):
        twobit = bit6[0] + bit6[-1]
        return twobit
    
    def get_middle_bit(self,bit6):
        fourbit = bit6[1:5]
        return fourbit
    
    #---------------------------
    def generate_f(self):
        after_s = ''
        #expansion
        expan = self.expansion48()
        XOR_48bit = self.XOR(expan,self.key_48bit)
                
        #s-box
        rawlist6,n = self.split_in_6bit(XOR_48bit), 0
        
        for i in rawlist6:
            '''binary to decimal'''
            row = int(self.get_1and6bit(i),2)
            col = int(self.get_middle_bit(i),2)
            value = self.s_box[n][row][col]
            '''decimal to binary 4bits a group'''
            self.sbox_list.append(bin(value)[2:].zfill(4))
            n = n + 1
        #permutation
        after_s = after_s.join(self.sbox_list)
        for index in self.straight_box:
            self.final32bit = self.final32bit + after_s[index-1]
            
        return self.final32bit  

#-------------------------------------------------------
bit32 = '00010011001101000101011101111001'  
bit48 = '110010110011110110001011000011100001011111110101'
s = mangler_func(bit32,bit48)   
s.generate_f()

'10001100010001000111011001001011'